In [37]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.weightstats import zconfint
import statsmodels.stats.multitest as smm

In [4]:
data = pd.read_csv('ab_browser_test.csv')
data.head()

,userID,browser,slot,n_clicks,n_queries,n_nonclk_queries
0,1,Browser #2,exp,23,32,19
1,3,Browser #4,exp,3,4,2
2,5,Browser #4,exp,29,35,16
3,6,Browser #4,control,12,6,0
4,7,Browser #4,exp,54,68,30


In [5]:
exp_clicks = data[data.slot == 'exp'].n_clicks.sum()
control_clicks = data[data.slot == 'control'].n_clicks.sum()
print(round((exp_clicks/control_clicks - 1)*100, 3))

1.614


In [6]:
data[data.slot == 'exp'].count()

userID              281580
browser             281580
slot                281580
n_clicks            281580
n_queries           281580
n_nonclk_queries    281580
dtype: int64

In [7]:
data[data.slot == 'control'].count()

userID              284554
browser             284554
slot                284554
n_clicks            284554
n_queries           284554
n_nonclk_queries    284554
dtype: int64

In [14]:
stats.mannwhitneyu(data[data.slot == 'exp'].n_clicks, data[data.slot == 'control'].n_clicks)

MannwhitneyuResult(statistic=38940590204.0, pvalue=3.1236179006848145e-75)

In [23]:
np.random.seed(0)
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

exp_samples = get_bootstrap_samples(np.array(data[data.slot == 'exp'].n_clicks), 1000)
control_samples = get_bootstrap_samples(np.array(data[data.slot == 'control'].n_clicks), 1000)

In [17]:
np.array(data[data.slot == 'exp'].n_clicks)

array([23,  3, 29, ...,  4,  4,  3])

In [24]:
exp_means = np.mean(exp_samples, axis=1)
control_means = np.mean(control_samples, axis=1)

means = exp_means - control_means

zconfint(means)

(0.29854363324803357, 0.3051808665376836)

In [26]:
exp_medians = np.median(exp_samples, axis=1)
control_medians = np.median(control_samples, axis=1)

medians = exp_medians - control_medians

zconfint(medians)

(1.0, 1.0)

In [31]:
exp_samples.shape

(1000, 281580)

Разделите выборку на две части, соответствующие control и exp группам. Преобразуйте данные к виду, чтобы каждому пользователю соответствовало суммарное значение его кликов. С помощью критерия Манна-Уитни проверьте гипотезу о равенстве средних. Что можно сказать о получившемся значении достигаемого уровня значимости ? 

In [29]:
control = data[data.slot == 'control'][{'n_clicks', 'userID'}]
control_clicks = control.groupby(['userID']).sum()
control_clicks

exp = data[data.slot == 'exp'][{'n_clicks', 'userID'}]
exp_clicks = exp.groupby(['userID']).sum()
exp_clicks

stats.mannwhitneyu(control_clicks['n_clicks'], exp_clicks['n_clicks'])

MannwhitneyuResult(statistic=38901259929.0, pvalue=4.3471471887604393e-75)

Проверьте, для какого из браузеров наиболее сильно выражено отличие между количеством кликов в контрольной и экспериментальной группах.

Для этого примените для каждого из срезов (по каждому из уникальных значений столбца browser) критерий Манна-Уитни между control и exp группами и сделайте поправку Холма-Бонферрони на множественную проверку с \alpha=0.05α=0.05.

Какое заключение можно сделать исходя из полученных результатов ?

В качестве ответа введите количество незначимых изменений с точки зрения результатов, полученных после введения коррекции.

In [21]:
data.browser.value_counts()

Browser #4     156833
Browser #22    142905
Browser #2     110737
Browser #14     89123
Browser #20     33929
Browser #17     32607
Name: browser, dtype: int64

In [30]:
browser_4_exp = np.array(data[(data.slot == 'exp') & (data.browser == 'Browser #4')].n_clicks)
browser_22_exp = np.array(data[(data.slot == 'exp') & (data.browser == 'Browser #22')].n_clicks)
browser_2_exp = np.array(data[(data.slot == 'exp') & (data.browser == 'Browser #2')].n_clicks)
browser_14_exp = np.array(data[(data.slot == 'exp') & (data.browser == 'Browser #14')].n_clicks)
browser_20_exp = np.array(data[(data.slot == 'exp') & (data.browser == 'Browser #20')].n_clicks)
browser_17_exp = np.array(data[(data.slot == 'exp') & (data.browser == 'Browser #17')].n_clicks)

browser_4_control = np.array(data[(data.slot == 'control') & (data.browser == 'Browser #4')].n_clicks)
browser_22_control = np.array(data[(data.slot == 'control') & (data.browser == 'Browser #22')].n_clicks)
browser_2_control = np.array(data[(data.slot == 'control') & (data.browser == 'Browser #2')].n_clicks)
browser_14_control = np.array(data[(data.slot == 'control') & (data.browser == 'Browser #14')].n_clicks)
browser_20_control = np.array(data[(data.slot == 'control') & (data.browser == 'Browser #20')].n_clicks)
browser_17_control = np.array(data[(data.slot == 'control') & (data.browser == 'Browser #417')].n_clicks)

In [35]:
browser_4 = stats.mannwhitneyu(browser_4_exp, browser_4_control).pvalue
browser_22 = stats.mannwhitneyu(browser_22_exp, browser_22_control).pvalue
browser_2 = stats.mannwhitneyu(browser_2_exp, browser_2_control).pvalue
browser_14 = stats.mannwhitneyu(browser_14_exp, browser_14_control).pvalue
browser_20 = stats.mannwhitneyu(browser_20_exp, browser_20_control).pvalue
browser_17 = stats.mannwhitneyu(browser_17_exp, browser_17_control).pvalue

In [41]:
values = [browser_4, browser_22, browser_2, browser_14, browser_20, browser_17]
browser_holm = smm.multipletests(pvals=values,
                                 method='holm')
browser_holm[1]<0.05

array([False, False, False,  True, False,  True])

Для каждого браузера в каждой из двух групп (control и exp) посчитайте долю запросов, в которых пользователь не кликнул ни разу. Это можно сделать, поделив сумму значений n_nonclk_queries на сумму значений n_queries. Умножив это значение на 100, получим процент некликнутых запросов, который можно легче проинтерпретировать.

Сходятся ли результаты проведенного Вами анализа с показателем процента некликнутых запросов ? Отметьте все верные утверждения.

In [60]:
browser_exp_nonclick = data[data.slot == 'exp'][{'browser', 'n_nonclk_queries'}].groupby(['browser']).sum()
browser_exp_click = data[data.slot == 'exp'][{'browser', 'n_queries'}].groupby(['browser']).sum()

browser_control_nonclick = data[data.slot == 'control'][{'browser', 'n_nonclk_queries'}].groupby(['browser']).sum()
browser_control_click = data[data.slot == 'control'][{'browser', 'n_queries'}].groupby(['browser']).sum()

In [61]:
browser_exp = pd.concat([browser_exp_nonclick, browser_exp_click], sort=False, axis=1)
browser_exp

browser_control = pd.concat([browser_control_nonclick, browser_control_click], sort=False, axis=1)
browser_control

,n_nonclk_queries,n_queries
browser,,
Browser #14,161252,279998
Browser #17,75837,208921
Browser #2,260259,566239
Browser #20,78548,193752
Browser #22,315470,777135
Browser #4,498254,1060771


In [63]:
browser_exp['perc_exp'] = (browser_exp.n_nonclk_queries/browser_exp.n_queries)*100
browser_exp

,n_nonclk_queries,n_queries,perc,perc_exp
browser,,,,
Browser #14,123656,282606,43.755617,43.755617
Browser #17,77614,210123,36.937413,36.937413
Browser #2,246514,548019,44.982747,44.982747
Browser #20,72998,187283,38.977376,38.977376
Browser #22,296177,743156,39.853947,39.853947
Browser #4,456508,1011250,45.142942,45.142942


In [64]:
browser_control['perc_control'] = (browser_control.n_nonclk_queries/browser_control.n_queries)*100
browser_control

,n_nonclk_queries,n_queries,perc_control
browser,,,
Browser #14,161252,279998,57.590411
Browser #17,75837,208921,36.299367
Browser #2,260259,566239,45.962747
Browser #20,78548,193752,40.540485
Browser #22,315470,777135,40.593977
Browser #4,498254,1060771,46.970930


In [67]:
browser_result = pd.concat([browser_control, browser_exp.perc_exp], sort=False, axis=1)
browser_result

,n_nonclk_queries,n_queries,perc_control,perc_exp
browser,,,,
Browser #14,161252,279998,57.590411,43.755617
Browser #17,75837,208921,36.299367,36.937413
Browser #2,260259,566239,45.962747,44.982747
Browser #20,78548,193752,40.540485,38.977376
Browser #22,315470,777135,40.593977,39.853947
Browser #4,498254,1060771,46.970930,45.142942
